In [1]:
import pymongo

from datetime import datetime
from time import strptime
from pymongo import MongoClient

Make a connection

In [2]:
client = MongoClient(host='192.168.43.225')
client

MongoClient(host=['192.168.43.225:27017'], document_class=dict, tz_aware=False, connect=True)

In [3]:
db = client.soccerdb
db

Database(MongoClient(host=['192.168.43.225:27017'], document_class=dict, tz_aware=False, connect=True), 'soccerdb')

In [5]:
matches = db.matches

In [6]:
matches.find_one()

{'_id': ObjectId('5d9dd62d1c801fea8e2419b7'),
 'competitionId': 364,
 'date': 'May 13, 2018 at 4:00:00 PM GMT+2',
 'dateutc': '2018-05-13 14:00:00',
 'duration': 'Regular',
 'gameweek': 38,
 'label': 'Burnley - AFC Bournemouth, 1 - 2',
 'referees': [{'refereeId': 385705, 'role': 'referee'},
  {'refereeId': 385733, 'role': 'firstAssistant'},
  {'refereeId': 385922, 'role': 'secondAssistant'},
  {'refereeId': 388333, 'role': 'fourthOfficial'}],
 'roundId': 4405654,
 'seasonId': 181150,
 'status': 'Played',
 'teamsData': {'1646': {'coachId': 8880,
   'formation': {'bench': [{'goals': 'null',
      'ownGoals': '0',
      'playerId': 77502,
      'redCards': '0',
      'yellowCards': '0'},
     {'goals': 'null',
      'ownGoals': '0',
      'playerId': 270828,
      'redCards': '0',
      'yellowCards': '0'},
     {'goals': 'null',
      'ownGoals': '0',
      'playerId': 9164,
      'redCards': '0',
      'yellowCards': '0'},
     {'goals': 'null',
      'ownGoals': '0',
      'playerId': 

In [7]:
a_match = matches.find_one({}, {'teamsData': 1, '_id': 0})

In [8]:
list(a_match['teamsData'].keys())[0]

'1659'

In [9]:
a_match['teamsData']['1646']['score']

1

In [10]:
for match in db.matches.find({}, {'teamsData': 1, '_id': 0}):
    team_ids = list(match['teamsData'].keys())
    for i in range(len(team_ids)):
        print(team_ids[i],match['teamsData'][team_ids[i]]['score'])

1659 2
1646 1
1628 2
1627 0
1609 1
1673 0
1612 4
1651 0
1644 0
1611 1
1613 3
1610 0
1619 0
1625 1
10531 1
1639 2
1631 4
1624 5
1623 1
1633 3
1633 0
1611 0
1651 1
1625 3
1613 0
1624 1
1631 3
1609 1
1610 1
1673 1
10531 0
1619 1
1609 5
1646 0
1612 0
1610 1
1673 0
1625 0
1623 1
1619 1
10531 0
1659 1
1631 0
1633 2
1644 2
1613 1
1624 0
1627 1
1628 2
1639 1
1651 1
1611 0
1644 0
1624 2
1609 1
1611 2
1633 1
1625 4
10531 0
1610 1
1646 0
1651 0
1628 5
1631 0
1623 2
1673 0
1613 0
1627 1
1659 1
1619 2
1612 0
1639 0
1623 1
1613 0
10531 0
1625 5
1609 4
1633 1
1639 1
1646 1
1628 0
1644 0
1612 2
1627 2
1610 2
1646 1
1631 0
1619 0
1659 0
1611 2
1624 1
1651 1
1633 1
1639 1
1627 1
1611 0
1613 2
1609 1
1624 1
1625 3
1612 3
1659 0
1631 1
1646 2
1628 3
1651 2
1644 0
1673 1
1623 1
10531 1
1610 3
1619 2
1610 1
1633 1
1609 3
1619 2
1611 3
1625 2
1628 2
1659 2
1651 1
1673 1
1631 1
1613 2
1624 2
1639 1
1644 1
1646 2
10531 1
1627 1
1623 0
1612 0
1624 3
1610 1
1609 3
1639 0
1623 1
1625 3
1633 3
1619 0
1631 2
1651 0

In [11]:
a_match = db.events.find_one()
print(a_match)

{'matchPeriod': '1H', 'positions': [{'x': 50, 'y': 48}, {'x': 47, 'y': 50}], 'subEventId': 85, 'eventName': 'Pass', 'tags': [{'id': 1801}], 'eventId': 8, 'matchId': 1694390, 'playerId': 26010, 'eventSec': 1.255989999999997, '_id': ObjectId('5d9dced91c801fea8e22e72e'), 'id': 88178642, 'teamId': 4418, 'subEventName': 'Simple pass'}


In [12]:
results = list(db.events.find({},{'matchId': 1, 'teamId': 1, '_id': 0}))

In [13]:
seen = set()
result_score = []
for d in results:
    t = tuple(d.items())
    if t not in seen:
        seen.add(t)
        result_score.append(d)
for d in result_score:
    d['score'] = 0
for match in db.matches.find({}, {'_id': 0, 'wyId': 1, 'date': 1}):
    for d in result_score:
        if(d['matchId'] == match['wyId']):
            d['date'] = match['date']
print(result_score)

[{'date': 'June 10, 2016 at 9:00:00 PM GMT+2', 'teamId': 4418, 'matchId': 1694390, 'score': 0}, {'date': 'June 10, 2016 at 9:00:00 PM GMT+2', 'teamId': 11944, 'matchId': 1694390, 'score': 0}, {'date': 'June 11, 2016 at 3:00:00 PM GMT+2', 'teamId': 8731, 'matchId': 1694391, 'score': 0}, {'date': 'June 11, 2016 at 3:00:00 PM GMT+2', 'teamId': 6697, 'matchId': 1694391, 'score': 0}, {'date': 'June 15, 2016 at 6:00:00 PM GMT+2', 'teamId': 6697, 'matchId': 1694392, 'score': 0}, {'date': 'June 15, 2016 at 6:00:00 PM GMT+2', 'teamId': 11944, 'matchId': 1694392, 'score': 0}, {'date': 'June 15, 2016 at 9:00:00 PM GMT+2', 'teamId': 8731, 'matchId': 1694393, 'score': 0}, {'date': 'June 15, 2016 at 9:00:00 PM GMT+2', 'teamId': 4418, 'matchId': 1694393, 'score': 0}, {'date': 'June 19, 2016 at 9:00:00 PM GMT+2', 'teamId': 4418, 'matchId': 1694394, 'score': 0}, {'date': 'June 19, 2016 at 9:00:00 PM GMT+2', 'teamId': 6697, 'matchId': 1694394, 'score': 0}, {'date': 'June 19, 2016 at 9:00:00 PM GMT+2', '

In [25]:
for match in db.matches.find({}, {'teamsData': 1, '_id': 0, 'wyId': 1, 'date': 1}):
    team_ids = list(match['teamsData'].keys())
    for i in range(len(team_ids)):
        for result in result_score:
            try:
                if (result["matchId"] == match['wyId']
                    and datetime.strptime(result['date'], "%B %d, %Y at %H:%M:%S %p %Z") < datetime.strptime(match['date'], "%B %d, %Y at %H:%M:%S %p %Z")
                    and result['teamId'] == match['teamsData'][team_ids[i]]['teamId']):
                    #result['score'] = result['score'] + match['teamsData'][team_ids[i]]['score']
                    print("yes")
                    break
            except:
                print(result['date'])
#print(result_score)

July 10, 2016 at 9:00:00 PM GMT+2
July 10, 2016 at 9:00:00 PM GMT+2
July 10, 2016 at 9:00:00 PM GMT+2
July 10, 2016 at 9:00:00 PM GMT+2
July 7, 2016 at 9:00:00 PM GMT+2
July 7, 2016 at 9:00:00 PM GMT+2
July 7, 2016 at 9:00:00 PM GMT+2
July 7, 2016 at 9:00:00 PM GMT+2
July 6, 2016 at 9:00:00 PM GMT+2
July 6, 2016 at 9:00:00 PM GMT+2
July 6, 2016 at 9:00:00 PM GMT+2
July 6, 2016 at 9:00:00 PM GMT+2
July 3, 2016 at 9:00:00 PM GMT+2
July 3, 2016 at 9:00:00 PM GMT+2
July 3, 2016 at 9:00:00 PM GMT+2
July 3, 2016 at 9:00:00 PM GMT+2
July 2, 2016 at 9:00:00 PM GMT+2
July 2, 2016 at 9:00:00 PM GMT+2
July 2, 2016 at 9:00:00 PM GMT+2
July 2, 2016 at 9:00:00 PM GMT+2
July 1, 2016 at 9:00:00 PM GMT+2
July 1, 2016 at 9:00:00 PM GMT+2
July 1, 2016 at 9:00:00 PM GMT+2
July 1, 2016 at 9:00:00 PM GMT+2
June 30, 2016 at 9:00:00 PM GMT+2
June 30, 2016 at 9:00:00 PM GMT+2
June 30, 2016 at 9:00:00 PM GMT+2
June 30, 2016 at 9:00:00 PM GMT+2
June 27, 2016 at 9:00:00 PM GMT+2
June 27, 2016 at 9:00:00 PM GMT+2


In [38]:
from dateutil import parser
s = 'July 7, 2016 at 9:00:00 PM GMT+2'
a = parser.parse(s)
for match in db.matches.find({}, {'teamsData': 1, '_id': 0, 'wyId': 1, 'date': 1}):
    team_ids = list(match['teamsData'].keys())
    for i in range(len(team_ids)):
        for result in result_score:
            try:
                if (result["matchId"] == match['wyId']
                    and parser.parse(result['date']) < parser.parse(match['date'])
                    and result['teamId'] == match['teamsData'][team_ids[i]]['teamId']):
                    #result['score'] = result['score'] + match['teamsData'][team_ids[i]]['score']
                        print("yes")
                        break
            except:
                print(result['date'])
#print(result_score)